
### **Chapter 2: Linear Quadratic Regulator**


In this chapter, we will provide the implementation of the LQR controller and demonstrate how it can be applied to a specific task (stabilization task). We begin by formulating the finite-horizon discrete-time LQR, where the optimal control policy is computed via backward dynamic programming over a fixed time window. This formulation provides time-varying feedback gains and serves as a natural starting point for understanding the structure of optimal control.

We then move to the infinite-horizon LQR formulation, which yields a time-invariant feedback gain by solving the algebraic Riccati equation (DARE). This approach is more computationally efficient and widely used in practice when the control objective is to stabilize the system over an indefinite time horizon.

Finally, we conclude by discussing several limitations of the LQR framework, including its reliance on linearized dynamics, quadratic cost assumptions, and not compatible with constraints. These limitations motivate the exploration of more advanced control methods, such as Model Predictive Control (MPC) and reinforcement learning-based approaches.

All the contents are summarized in the table below.  

<table border="1" style="border-collapse: collapse; text-align: center;">
  <!-- Title Row -->
  <tr>
    <th colspan="2" style="text-align:center">Content of Chapter 2 Exercise</th>
  </tr>

  <!-- Row group 1 -->
  <tr>
    <td rowspan="2">Finite-horizon LQR</td>
    <td>implement the finite-horizon LQR controller using bellman recursion</td>
  </tr>
  <tr>
    <td>simulation and visualization</td>
  </tr>

  <!-- Row group 2 -->
  <tr>
    <td rowspan="2">Infinite-horizon LQR</td>
    <td>implement the infinite-horizon LQR controller using bellman equation</td>
  </tr>
  <tr>
    <td>simulation and visualization</td>
  </tr>

  <!-- Row group 3 -->
  <tr>
    <td rowspan="1">Influence of hyperparameters in LQR</td>
    <td>analyze the impact of adjusting the weight matrices on system behavior.</td>
  </tr>

  <!-- Row group 4 -->
  <tr>
    <td rowspan="2">Limitations of LQR</td>
    <td>Limitation 1: linear dynamics and quadratic cost</td>
  </tr>
  <tr>
    <td>Limitation 2: not accounting for constraints</td>
  </tr>

</table>

First, we need to set up our Python environment and import relevant packages.

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from rest.utils import *

import numpy as np
import casadi as ca

### **Problem setup:**

- Task: start from given initial position $p_0$, to reach a given terget position $p_T$ (Stabilization)

- Slope profile (height $h$ with reference to horizontal displacement $p$):  
   - case 1: zero slope (linear case), $h(p) = c$
   - case 2: constant slope (linear case), $h(p) = \frac{\pi}{18} \cdot p$
   - case 3: varying slope for small disturbances (nonlinear case), $h(p) = k \cdot \cos(18 p)$
   - case 4: varying slope for under actuated case (nonlinear case), $h(p) = \begin{cases} k \cdot \sin(3 p), & p \in [- \frac{\pi}{2}, \frac{\pi}{6}] \\ k, & p \in (-\infty, -\frac{\pi}{2}) \cup (\frac{\pi}{6}, \infty) \end{cases}$

- System dynmaics of 1d mountain car model (in State space representation): 
   - state vector $\boldsymbol{x} = [p, v]^T$
   - input vector $u$
   - system dynamics:
   \begin{align*}
     \begin{bmatrix} \dot{p} \\ \dot{v} \end{bmatrix} = \begin{bmatrix} v \\ - g \sin(\theta) \cos(\theta) \end{bmatrix} + \begin{bmatrix} 0 \\ \cos(\theta)  \end{bmatrix} u
   \end{align*}


### **Preparation: define the mountain car environment and the system dynamics**

In the previous exercise, we demonstrated how to define a symbolic function using CasADi symbolic system, inclusive defining the profile over a slope $h(p)$, deriving the conversion formulas between the slope profile $h(p)$ and the inclination angle $\theta(p)$, and establish the system's dynamics. These formulas have already been integrated into the class `Env` and `Dynamics`. In this chapter, we will specify the arguments and instantiate these classes directly to utilize their functionalities.

**Step 1: specify the arguments for class `Env` and instantiate the class**

- To start with the simpler case (also more compatible with LQR), we will initially focus on a linear system in an unconstrained scenario

- Parameters in the task:  
   - case: 1 (linear case)
   
   - initial state: $\boldsymbol{x}_0 = [-0.5, 0.0]^T$
   - target state: $\boldsymbol{x}_T = [0.6, 0.0]^T$

**Step 2: call function `test_env()` to plot the mountain profile $h(p)$ and curve of inclination angle $\theta(p)$**

**Step 3: specify the arguments for class `Dynmaics` and instantiate the class**



In [ ]:
# Define profile of slope, the initial / target state
case = 1 # 1 or 2
initial_position = -0.5
initial_velocity = 0.0
target_position = 0.6
target_velocity = 0.0

# Instantiate class 'Env'
# Arguments (without constraints): 
#   1) case: $n \in [1, 2, 3, 4]$, type: int
#   2) initial state: x_0 = [p_0, v_0], type: np.array
#   3) terminal state: x_T = [p_T, v_T], type: np.array
env = Env(case, np.array([initial_position, initial_velocity]), np.array([target_position, target_velocity]))
env.test_env() #  shape of slope (left side) and theta curve (right side) 

# Instantiate class 'Dynamics'
# Arguments: 
#   1) an object of class `Env`, type: Env  
dynamics = Dynamics(env)

---

### **Part (a): finite-horizon discrete-time LQR controller**

In this section, we will provide the implementation of the finite horizon LQR controller and demonstrate how it can be applied to a specific task (stabilization task). 

**Problem formulation:**

For a linear system:

$$
\boldsymbol{x_{k+1}} = \boldsymbol{A} \boldsymbol{x_k} + \boldsymbol{B} \boldsymbol{u_k}
$$

The finite-horizon quadratic cost function is given by:

$$
J(\boldsymbol{x_0}) = \boldsymbol{x}_N^T \boldsymbol{Q}_N \boldsymbol{x}_N + \sum_{k=0}^{N-1} \left(\boldsymbol{x}_k^T \boldsymbol{Q}_k \boldsymbol{x}_k + \boldsymbol{u}_k^T \boldsymbol{R}_k \boldsymbol{u}_k\right)
$$

*Note that: in this formulation, the weighting matrices $\boldsymbol{Q}_k$ and $\boldsymbol{R}_k$ are defined as time-varying. This is intended to maintain generality in the problem setup.
In practical applications, however, it is common to assume time-invariant weights, that is, $\boldsymbol{Q}_0=...=\boldsymbol{Q}_{N}=\boldsymbol{Q}$ and $\boldsymbol{R}_0=...=\boldsymbol{R}_{N-1}=\boldsymbol{R}$, which can also be regarded as a special case of this general formulation.*

**Problem solving:**

We solve this problem using **dynamic programming**, starting from the terminal time step $ k = N $ and working backwards to $ k = 0 $.

We start with the value function at time $N$:

$$
V_N(\boldsymbol{x}_N) = \boldsymbol{x}_N^T \boldsymbol{Q}_N \boldsymbol{x}_N
$$

with terminal condition:

$$
\boldsymbol{S}_N = \boldsymbol{Q}_N
$$

The **Bellman recursion** for backward value function update is given by:

$$
\begin{aligned}
V_k(\boldsymbol{x}_k) 
&= \min_{\boldsymbol{u}_k} \left[ \boldsymbol{x}_k^T \boldsymbol{Q}_k \boldsymbol{x}_k + \boldsymbol{u}_k^T \boldsymbol{R}_k \boldsymbol{u}_k + V_{k+1}(\boldsymbol{x}_{k+1}) \right] \\
&= \min_{\boldsymbol{u}_k} \left[ \boldsymbol{x}_k^T \boldsymbol{Q}_k \boldsymbol{x}_k + \boldsymbol{u}_k^T \boldsymbol{R}_k \boldsymbol{u}_k + (\boldsymbol{A} \boldsymbol{x}_k + \boldsymbol{B} \boldsymbol{u}_k)^T \boldsymbol{S}_{k+1} (\boldsymbol{A} \boldsymbol{x}_k + \boldsymbol{B} \boldsymbol{u}_k) \right]
\end{aligned}
$$

Expanding the above and completing the square yields the optimal control law:

$$
\boldsymbol{u}_k^* = \boldsymbol{K}_k \boldsymbol{x}_k 
$$

with time-varying gain matrix:

$$
\boldsymbol{K}_k = -(\boldsymbol{R}_k + \boldsymbol{B}^T \boldsymbol{S}_{k+1} \boldsymbol{B})^{-1} \boldsymbol{B}^T \boldsymbol{S}_{k+1} \boldsymbol{A}
$$

The Riccati matrix update, also known as **discrete-time algebraic Riccati equation (DARE)**, is:

$$
\boldsymbol{S}_k = \boldsymbol{Q}_k + \boldsymbol{A}^T \boldsymbol{S}_{k+1} \boldsymbol{A} - \boldsymbol{A}^T \boldsymbol{S}_{k+1} \boldsymbol{B} (\boldsymbol{R}_k + \boldsymbol{B}^T \boldsymbol{S}_{k+1} \boldsymbol{B})^{-1} \boldsymbol{B}^T \boldsymbol{S}_{k+1} \boldsymbol{A}
$$


**Step 1: implement the finite horizon LQR controller**  

In this step, we will implement the setup function `setup_external()`, which will be automatically called in the constructor of class `FiniteLQRController`. The implementation will follow three key steps:

1\) Obtain the system matrices: Retrieve the $\boldsymbol{A}$ and $\boldsymbol{B}$ matrices from the discrete-time system dynamics;  
    Hint: you may use the method `get_linearized_AB_discrete()` from class `Dynamics` to get the discretized system matrices

2\) Calculate the feedback gain $\boldsymbol{K}$: Use the computed Hessian $\boldsymbol{S}$ to determine the LQR feedback gain $\boldsymbol{K}$;  
    Hint: you may use the method `numpy.linalg.inv()` to solve the inverse matrix

3\) Update the Hessian matrix $\boldsymbol{S}$ of optimal cost: use riccati recursion to calculate $\boldsymbol{S}$ for the last step;  

In [ ]:
def setup_external(self) -> None:
        
    # Set up equilibrium state
    self.x_eq = self.state_lin

    # Solve input at equilibrium
    self.u_eq = self.dynamics.get_equilibrium_input(self.x_eq)

    # Linearize and discretize the system dynamics at equilibrium
    self.A, self.B = self.dynamics.get_linearized_AB_discrete(
        current_state=self.x_eq, current_input=self.u_eq, dt=self.dt
    )

    # Initialize terminal cost
    S = self.Q_N.copy()

    # Solve Bellman Recursion from backwards to compute gain matrix
    for k in reversed(range(self.N)):

        K = - np.linalg.inv(self.R + self.B.T @ S @ self.B) @ (self.B.T @ S @ self.A)

        self.K_list[k] = K

        S = self.Q + self.A.T @ S @ self.A - (self.A.T @ S @ self.B) @ np.linalg.inv(self.R + self.B.T @ S @ self.B) @ (self.B.T @ S @ self.A)

    if self.verbose:
        print(f"LQR Gain Matrix K: {self.K}")

**Step 2: Bind the defined setup function to the class "FiniteLQRController", and run the simulation to see the performance of controller**  

1\) Bind the defined setup function for DP algorithm `setup_external()` to class `FiniteLQRController`;

2\) Specify the arguments and instantiate the controller class `FiniteLQRController`; 

- Parameters in the task:  

    i) weight for state $\boldsymbol{Q} = \text{diag}([1, 1])$ (requirement: symmetric, positive semi-definite matrix)  

    ii) weight for input $\boldsymbol{R} = [0.1]$ (requirement: symmetric, positive definite matrix)  
    
    iii) control frequency $f = 20$

3\) Instantiate the class `Simulator` and call function `run_simulation()` to generate the simulated state- and input-trajectory;

4\) Instantiate the class `Visualizer`, call function `display_final_results()` and `display_animation()` to show the simulations;


In [ ]:
# Bind the defined LQR algorithm to the corresponding class, will be automatically called by constructor
FiniteLQRController.setup = setup_external

# Define weight matrix in stage and terminal cost
Q = np.diag([1, 1])
R = np.array([[0.1]])
Q_N = Q

# Define parameters of simulation
freq = 20 # controll frequency
t_terminal = 10 # time length of simulation
horizon = freq * t_terminal # number of time steps

# Instantiate the finite horizon LQR controller class
# Arguments: 
#   1) an object of class `Env` (to deliver infos about initial state, constraints, etc.), type: Env  
#   2) an object of class `Dynamics` (to deliver infos about symbolic system dynamics), type: Dynamics  
#   3) weight matrices in cost functions: i) `Q`: weight metrix of state, type: np.array  
#                                         ii) `R`: weight metrix of input, type: np.array  
#                                         iii) `Q_N`: weight metrix of state, type: np.array  
#   4) freq: control frequency $f$ , type: int  
#   5) name: the name of current coltroller displayed in plots, type: string
controller_lqr_f = FiniteLQRController(env, dynamics, Q, R, Q_N, freq, horizon=horizon, name='LQR_finite')

# Instantiate the simulator, and then run the simulation
simulator_lqr_f = Simulator(dynamics, controller_lqr_f, env, 1/freq, t_terminal)
simulator_lqr_f.run_simulation()

# Instantiate the visualizer, and display the plottings and animation
visualizer_lqr_f = Visualizer(simulator_lqr_f)
visualizer_lqr_f.display_plots()
visualizer_lqr_f.display_animation()

---

### **Part (b): infinite-horizon discrete-time LQR controller**

While the finite-horizon LQR offers greater flexibility by incorporating terminal costs and producing time-varying control gains, it suffers from a key limitation: the solution can only be computed via backward recursion. As a result, the entire control policy must be precomputed before execution, making it unsuitable for online adaptation in most practical scenarios. In this section, we will provide the implementation of the LQR controller and demonstrate how it can be applied to a specific task (stabilization task). We will also show the resulting performance and the effects of using the LQR controller in achieving optimal control.


**Problem formulation:**

For a same linear system formulation:

$$
\boldsymbol{x_{k+1}} = \boldsymbol{A} \boldsymbol{x_k} + \boldsymbol{B} \boldsymbol{u_k}
$$

As the horizon length $N$ approaches infinity, the problem converges to the infinite-horizon formulation with time-invariant optimal policy. Then the infinite-horizon quadratic cost function is given by:

$$
J(\boldsymbol{x_0}) = \sum_{k=0}^{\infty} \left(\boldsymbol{x}_k^T \boldsymbol{Q} \boldsymbol{x}_k + \boldsymbol{u}_k^T \boldsymbol{R} \boldsymbol{u}_k\right)
$$

**Explicit Solution:**

The solution to this problem is obtained by solving the **Discrete Algebraic Riccati Equation (DARE):**

$$
\boldsymbol{S} = \boldsymbol{Q} + \boldsymbol{A}^T \boldsymbol{S} \boldsymbol{A} - \boldsymbol{A}^T \boldsymbol{S} \boldsymbol{B}(\boldsymbol{R} + \boldsymbol{B}^T \boldsymbol{S} \boldsymbol{B})^{-1} \boldsymbol{B}^T \boldsymbol{S} \boldsymbol{A}
$$

The optimal feedback control policy is given by:

$$
\boldsymbol{u^*} = \boldsymbol{K} \boldsymbol{x_k}, \quad \boldsymbol{K} = -\left( \boldsymbol{R} + \boldsymbol{B}^T \boldsymbol{S} \boldsymbol{B} \right)^{-1} \boldsymbol{B}^T \boldsymbol{S} \boldsymbol{A}
$$

**Step 1: implement the infinite horizon LQR controller**  

In this step, we will implement the setup function `setup_external()`, which will be automatically called in the constructor of class `LQRController`. The implementation will follow three key steps:

1\) Obtain the system matrices: Retrieve the $\boldsymbol{A}$ and $\boldsymbol{B}$ matrices from the discrete-time system dynamics;  
    Hint: you may use the method `get_linearized_AB_discrete()` from class `Dynamics` to get the discretized system matrices

2\) Solve for the Hessian matrix $\boldsymbol{S}$ of optimal cost: Compute the solution to the Discrete Algebraic Riccati Equation (DARE) to obtain $\boldsymbol{S}$;  
    Hint: you may use the method `scipy.linalg.solve_discrete_are()` to solve DARE

3\) Calculate the feedback gain $\boldsymbol{K}$: Use the computed Hessian $\boldsymbol{S}$ to determine the LQR feedback gain $\boldsymbol{K}$;  
    Hint: you may use the method `numpy.linalg.inv()` to solve the inverse matrix

In [ ]:
def setup_external(self):

    # Set up equilibrium state
    self.x_eq = self.state_lin # self.state_lin default to be target state

    # Solve input at equilibrium
    self.u_eq = self.dynamics.get_equilibrium_input(self.x_eq)

    # Linearize dynamics at equilibrium
    # Hint: use function self.dynamics.get_linearized_AB_discrete
    self.A, self.B = self.dynamics.get_linearized_AB_discrete(
        current_state=self.x_eq, current_input=self.u_eq, dt=self.dt
    )

    # Solve DARE to compute gain matrix
    # Hint: use function scipy.linalg.solve_discrete_are
    S = scipy.linalg.solve_discrete_are(self.A, self.B, self.Q, self.R)
    self.K = - np.linalg.inv(self.R + self.B.T @ S @ self.B) @ (self.B.T @ S @ self.A)

    # With this LQR gain self.K, the input can be calculate by multiplying the state difference
    # u = self.u_eq + self.K @ (current_state - self.self.x_eq)


**Step 2: Bind the defined setup function to the class "LQRController", and run the simulation to see the performance of controller**  

1\) Bind the defined setup function for DP algorithm `setup_external()` to class `LQRController`;

2\) Specify the arguments and instantiate the controller class `LQRController` as `LQR_Q=10R`; 

- Parameters in the task:  

    i) weight for state $\boldsymbol{Q} = \text{diag}([1, 1])$ (requirement: symmetric, positive semi-definite matrix)  

    ii) weight for input $\boldsymbol{R} = [0.1]$ (requirement: symmetric, positive definite matrix)  
    
    iii) control frequency $f = 20$

3\) Instantiate the class `Simulator` and call function `run_simulation()` to generate the simulated state- and input-trajectory;

4\) Instantiate the class `Visualizer`, call function `display_final_results()` and `display_animation()` to show the simulations;


In [ ]:
# Bind the defined DP algorithm to the corresponding class, will be automatically called by constructor
LQRController.setup = setup_external

# Define weight matrix in stage and terminal cost
Q_0 = np.diag([1, 1])
R = np.array([[0.1]])

# Define parameters of simulation
freq = 20 # controll frequency
t_terminal = 10 # time length of simulation

# Instantiate the LQR controller class
# Arguments: 
#   1) an object of class `Env` (to deliver infos about initial state, constraints, etc.), type: Env  
#   2) an object of class `Dynamics` (to deliver infos about symbolic system dynamics), type: Dynamics  
#   3) weight matrices in cost functions: i) `Q`: weight metrix of state, type: np.array  
#                                         ii) `R`: weight metrix of input, type: np.array  
#   4) freq: control frequency $f$ , type: int  
#   5) name: the name of current coltroller displayed in plots, type: string
controller_lqr_0 = LQRController(env, dynamics, Q_0, R, freq, name='LQR_Q=10R')

# Instantiate the simulator, and then run the simulation
simulator_lqr_0 = Simulator(dynamics, controller_lqr_0, env, 1/freq, t_terminal)
simulator_lqr_0.run_simulation()

# Instantiate the visualizer, and display the plottings and animation
visualizer_lqr_0 = Visualizer(simulator_lqr_0)
visualizer_lqr_0.display_plots()
visualizer_lqr_0.display_animation()


---

### **Part (C): Influence of the Weight Matrix on LQR Performance**

In this section, we will analyze how the choice of weight matrices in the cost function affects the performance of the LQR controller. By tuning the weight matrices $\boldsymbol{Q}$ (which penalize deviations in state) while keeping $\boldsymbol{R}$ (which penalizes control input) fixed, we can observe how the system behaves under different control strategies.

Specifically, we define two LQR controller with different weight matrices: 

- `LQR_Q=100R`: $ \quad \boldsymbol{Q}_1 = \text{diag}([10, 10]), \quad \boldsymbol{R} = [0.1]$  

- `LQR_Q=R`: $ \quad \boldsymbol{Q}_2 = \text{diag}([0.1, 0.1]), \quad \boldsymbol{R} = [0.1]$  

Using these configurations, we simulate the system's response under two LQR controllers and and make a contrast with the controller defined in the last step (`LQR_Q=10R`). By observing the results, we gain insight into how adjusting the weight matrix influences the controller's aggressiveness and efficiency.

*Note that: here we only vary the value of $\boldsymbol{Q}$. The effect of $\boldsymbol{R}$ is inverse, as the controller's behavior is fundamentally influenced by the relative magnitude of $\boldsymbol{Q}$ and $\boldsymbol{R}$.*

In [ ]:
# Define weight matrix in stage and terminal cost
#   for reference: Q_0 = np.diag([1, 1])
Q_1 = np.diag([10, 10])
Q_2 = np.diag([0.1, 0.1])

# Instantiate the LQR controller class, the simulator class, and then run the simulation
controller_lqr_1 = LQRController(env, dynamics, Q_1, R, freq, name='LQR_Q=100R')
simulator_lqr_1 = Simulator(dynamics, controller_lqr_1, env, 1/freq, t_terminal)
simulator_lqr_1.run_simulation()

controller_lqr_2 = LQRController(env, dynamics, Q_2, R, freq, name='LQR_Q=R')
simulator_lqr_2 = Simulator(dynamics, controller_lqr_2, env, 1/freq, t_terminal)
simulator_lqr_2.run_simulation()

# Instantiate the visualizer, and display the plottings and animation
visualizer_lqr_0.display_contrast_plots(simulator_lqr_1, simulator_lqr_2)
visualizer_lqr_0.display_contrast_animation_same(simulator_lqr_1, simulator_lqr_2)

#### **Results Analysis: Influence of LQR Weight Matrices on System Performance**

The figure above compares the performance of three Linear Quadratic Regulator (LQR) controllers: **LQR_Q_10**, **LQR_Q_1**, and **LQR_Q_01**. Each controller is defined with different weight matrices $\boldsymbol{Q}$, which penalize the state deviations, while $\boldsymbol{R}$, which penalizes control effort, remains the same.

The comparison clearly demonstrates the impact of the weight matrix $\boldsymbol{Q}$ on LQR performance:
1. **Higher $\boldsymbol{Q}$** (`LQR_Q=100R`):  
   - Prioritizes minimizing state deviations.  
   - Results in faster convergence but higher control effort.  

2. **Lower $\boldsymbol{Q}$** (`LQR_Q=R`):  
   - Reduces control effort and produces smoother trajectories.  
   - Leads to slower convergence and delayed state correction.  

3. **Moderate $\boldsymbol{Q}$** (`LQR_Q=10R`):  
   - Balances control effort and tracking performance.  

The selection of $\boldsymbol{Q}$ depends on the system's requirements:  
- For tasks requiring fast convergence, a higher $\boldsymbol{Q}$ is appropriate.  
- For energy-constrained systems, a lower $\boldsymbol{Q}$ ensures smoother and less aggressive control. 

#### **Main Conclusion:**

By tuning the relative magnitude of weight matrices $\boldsymbol{Q}$ and $\boldsymbol{R}$, LQR controllers can achieve the desired trade-off between tracking performance and control effort.

*Note that: in this example, we use $\boldsymbol{Q}$ with eigenvalues equal to $\boldsymbol{R}$ as a moderate baseline. However, this choice does not universally apply, as the magnitude of states and inputs depends on their units. But conversely, in the practice it is always important to carefully scale the states and inputs to similar numerical ranges. Otherwise, large discrepancies in the scales of $\boldsymbol{Q}$ and $\boldsymbol{R}$ can result in an ill-conditioned cost Hessian, causing numerical instability and divergence during optimization.*

---


### **Part (D): Limitations of the LQR Controller**

At last, we want to emphasize the limitations of LQR: 

  1) a **quadratic cost function**;

  2) a **linear dynamics**;  
  
  3) and operates under **no constraints**;  

While the quadratic cost is easy to be satisfied, we will mostly focus on the last 2 requirements.



#### **1. Linear vs. Nonlinear Dynamics**

LQR is designed based on linear system dynamics. However, real-world physical systems are often inherently nonlinear. If we directly apply an LQR controller to such nonlinear systems, what would happen? Will the controller still be able to achieve the control objectives, or will it completely fail to work? In this section, we will present simulation results under different scenarios to explore and discuss these questions.

It should first be noted that when attempting to apply an LQR control law to a nonlinear system, regardless of whether the controller ultimately works or not, we must first select an operating point around which to linearize the nonlinear dynamics. This requirement is inherent to the structure of the LQR design. The choice of the linearization point is arbitrary in principle, since at any given state we can always compute a corresponding equilibrium input by setting the right-hand side of the Newtonian dynamics to zero. However, in practice, **the selection of the linearization point can significantly affect the performance of the LQR controller when applied to a nonlinear system**, as will be demonstrated in the examples that follow.

The testbed was chosen to be case 4. The specific curve is shown in the figure below. It can be seen that it consists of a valley and a flat ground. The dynamics of the valley part is nonlinear (the inclination angle is a function of position $p$), while the flat ground part is linear.

In [ ]:
# Define the case
case_nonlinear = 4
# Instantiate class 'Env' 
env_nonlinear = Env(case_nonlinear, np.array([initial_position, initial_velocity]), np.array([target_position, target_velocity]))
env_nonlinear.show_slope()



##### **Example 1: linearize around the flat ground, drive the car from the top to the valley**

Let us first consider a relatively simple case: the car starts from flat ground and attempts to stabilize at the bottom of the valley under the action of the LQR control law, which implies that:

$$
p_{start} = 0.6, \quad p_{target} = -0.5,
$$

Considering that the valley itself is an attractive equilibrium point, the terminal condition is easily satisfied, and thus the linearization point can be directly chosen at the flat ground:

$$
\boldsymbol{\overline{x}} = [1.0, 0.0],
$$

Please run the following simulation and observe the results.


In [ ]:
# Define the start point and the target
initial_position = 0.6
target_position = -0.5

# Define linearization of the flat space
state_lin = np.array([1.0, 0.0])

# Instantiate class 'Env' 
env_nonlinear = Env(case_nonlinear, np.array([initial_position, initial_velocity]), np.array([target_position, target_velocity]))
# Instantiate class 'Dynamics'
dynamics_nonlinear = Dynamics(env_nonlinear)
# Instantiate the LQR controller class
controller_lqr_nonlinear = LQRController(env_nonlinear, dynamics_nonlinear, Q_1, R, freq, name='LQR_case4')
# Refresh the algorithm
controller_lqr_nonlinear.set_lin_point(state_lin)
# Instantiate the simulator, and then run the simulation
simulator_lqr_nonlinear = Simulator(dynamics_nonlinear, controller_lqr_nonlinear, env_nonlinear, 1/freq, t_terminal)
simulator_lqr_nonlinear.run_simulation()
# Instantiate the visualizer, and display the plottings and animation
visualizer_lqr_nonlinear = Visualizer(simulator_lqr_nonlinear)
visualizer_lqr_nonlinear.display_plots()
visualizer_lqr_nonlinear.display_animation()



##### **Example 2: linearize around the flat ground, drive the car from the valley to the top**

Let us slightly increase the difficulty of the task: while still choosing the flat ground as the linearization point, we attempt to use the LQR control law to drive the car from the bottom of the valley toward the right hilltop. The parameters can be set according to the following values:

$$
p_{start} = -0.5, \quad p_{target} = 0.6, \quad \boldsymbol{\overline{x}} = [1.0, 0.0],
$$

Please run the following simulation and see what will happen.

In [ ]:
# Define the start point and the target
initial_position = -0.5
target_position = 0.6

#Define linearization of the flat space
state_lin = np.array([1.0, 0.0])

# Instantiate class 'Env' 
env_nonlinear = Env(case_nonlinear, np.array([initial_position, initial_velocity]), np.array([target_position, target_velocity]))
# Instantiate class 'Dynamics'
dynamics_nonlinear = Dynamics(env_nonlinear)
# Instantiate the LQR controller class
controller_lqr_nonlinear = LQRController(env_nonlinear, dynamics_nonlinear, Q_1, R, freq, name='LQR_case4')
# Refresh the algorithm
controller_lqr_nonlinear.set_lin_point(state_lin)
# Instantiate the simulator, and then run the simulation
simulator_lqr_nonlinear = Simulator(dynamics_nonlinear, controller_lqr_nonlinear, env_nonlinear, 1/freq, t_terminal)
simulator_lqr_nonlinear.run_simulation()
# Instantiate the visualizer, and display the plottings and animation
visualizer_lqr_nonlinear = Visualizer(simulator_lqr_nonlinear)
visualizer_lqr_nonlinear.display_plots()
visualizer_lqr_nonlinear.display_animation()



##### **Example 3: linearize around a point on the slope, drive the car from the valley to the top**

Based on the simulation results of Example 2, we attempt to make some adjustments to the linearization point to observe how it affects the system's response. We move the linearization point to a point on the slope, which implies that:

$$
p_{start} = -0.5, \quad p_{target} = 0.6, \quad \boldsymbol{\overline{x}} = [0.0, 0.0],
$$

Rerun the simulation and observe the results.

In [ ]:
# Define the start point and the target
initial_position = -0.5
target_position = 0.6

#Define linearization of the flat space
state_lin = np.array([0.0, 0.0])

# Instantiate class 'Env' 
env_nonlinear = Env(case_nonlinear, np.array([initial_position, initial_velocity]), np.array([target_position, target_velocity]))
# Instantiate class 'Dynamics'
dynamics_nonlinear = Dynamics(env_nonlinear)
# Instantiate the LQR controller class
controller_lqr_nonlinear = LQRController(env_nonlinear, dynamics_nonlinear, Q_1, R, freq, name='LQR_case4')
# Refresh the algorithm
controller_lqr_nonlinear.set_lin_point(state_lin)
# Instantiate the simulator, and then run the simulation
simulator_lqr_nonlinear = Simulator(dynamics_nonlinear, controller_lqr_nonlinear, env_nonlinear, 1/freq, t_terminal)
simulator_lqr_nonlinear.run_simulation()
# Instantiate the visualizer, and display the plottings and animation
visualizer_lqr_nonlinear = Visualizer(simulator_lqr_nonlinear)
visualizer_lqr_nonlinear.display_plots()
visualizer_lqr_nonlinear.display_animation()

##### **Results Analysis: influence of the linearization point**

Summarizing the above three examples, we observe that both the task target and the choice of linearization point affect the performance of applying LQR to a nonlinear system. Specifically:

- **Example 1**: When the car moves downhill to the left, the required driving force can be fully provided by the LQR policy, thus allowing leftward movement. Moreover, the target itself is a stable equilibrium point of the nonlinear system, meaning that even in the presence of nonlinear disturbances, the system may exhibit some overshoot but will ultimately stabilize at the target point.  

- **Example 2**: Unlike in Example 1, when the car moves uphill along the slope, it becomes stuck halfway due to the insufficient input provided by the LQR policy. This is because the linearization point is chosen on the flat platform, leading the LQR controller to compute a policy that only generates inputs sufficient for horizontal motion. However, in order for the car to move uphill, additional input is required to overcome the gravitational force. As a result, the car is unable to generate enough input and ultimately gets stuck halfway up the slope.  

- **Example 3**: Building upon Example 2, we move the linearization point onto the slope, allowing the LQR controller to recognize the need to generate additional input to counteract the gravitational component when computing the policy. As a result, the car is able to climb up the slope. However, a drawback is that the car cannot be precisely stabilized at the target point, and a noticeable steady-state error appears. This can be interpreted as a loss of control accuracy caused by model mismatch.  

In summary, although the LQR policy can be applied to nonlinear systems, its performance cannot be guaranteed. Therefore, when making such attempts, it is crucial to **carefully select the linearization point** to improve performance as much as possible and to mitigate the losses caused by model mismatch.


*Note: The loss of accuracy in nonlinear systems can be mitigated by applying the **iterative LQR (iLQR)** method (which will be introduced in the next Lecture). The key idea of iLQR is to perform a Taylor expansion of the cost function and dynamics around each operating point, incorporating higher-order derivatives into the optimization process. In fact, LQR can be viewed as a special case of iLQR when the system dynamics are linear.*

<br>

<blockquote style="padding-top: 20px; padding-bottom: 10px;">

##### **❓ Extension Questions: Overshoot when applying LQR to nonlinear cases**

When applying LQR to nonlinear systems, it is common to observe that the resulting trajectory exhibits overshoot or even fails to reach the desired goal. In some cases, the system may stabilize at a steady-state far from the target.

**Open question 1**: What might cause this steady-state offset when applying an LQR controller to a nonlinear system?

*Hint: Think about the **assumptions** behind LQR design. LQR is derived from **linearized dynamics around an operating point**.*

**Open question 2**: How can you compute this offset? 

*Hint: Compute the steady-state of the closed-loop system. In this case you should use the nonlinear dynamics and substitute the LQR policy $\boldsymbol{u} = \boldsymbol{K} \boldsymbol{x}$ for the input. For the steady-state the velocity $v_{\text{steady}} = 0$. Then compute the position $p_{\text{steady}}$.*

</blockquote> 

<br>


##### **🔍 Question:**

For Example 3, is there a way to design the LQR such that the car can move uphill along the slope and precisely stop at the target point without exhibiting any steady-state error?

*Hint: think about a time-varying policy*



----


#### **2. Effect of Input Constraints on LQR Performance**

LQR is formulated under the assumption of **unconstrained inputs**, meaning it does not inherently support input saturation or upper/lower constraints. However, in real-world systems, control inputs such as force, torque, or voltage often have physical limits. When the LQR controller generates control inputs exceeding these limits, the inputs will be **clipped** to the boundaries. This behavior can cause saturation between the actual system trajectory and the ideal trajectory predicted by the LQR controller.

To illustrate this limitation, we design two test scenarios:

- **Ideal LQR Control (`LQR_linear`)**:  

   - Linear system 

   - No constraints are applied to the control inputs.

- **Clipped LQR Control (`LQR_linear_clipped`)**: 

   - Linear system 

   - Input constraints are imposed, which means that the input is limited within an upper and lower bound

   - Any control input exceeding these bounds is **clipped**, leading to saturation

In [ ]:
# Define the case
case_linear = 1

# Instantiate class 'Env' 
env_linear = Env(case_linear, np.array([initial_position, initial_velocity]), np.array([target_position, target_velocity]))
# Instantiate class 'Dynamics'
dynamics_linear = Dynamics(env_linear)
# Instantiate the LQR controller class
controller_lqr_linear = LQRController(env_linear, dynamics_linear, Q_0, R, freq, name='LQR_case1')
# Instantiate the simulator, and then run the simulation
simulator_lqr_linear = Simulator(dynamics_linear, controller_lqr_linear, env_linear, 1/freq, t_terminal)
simulator_lqr_linear.run_simulation()



# Define a upper bound of input a
input_ubs = 0.25
input_lbs = -0.25

# Instantiate class 'Env'
env_linear_clipped = Env(case_linear, np.array([initial_position, initial_velocity]), np.array([target_position, target_velocity]), 
                           input_lbs=input_lbs, input_ubs=input_ubs)
# Instantiate the LQR controller class
controller_lqr_linear_clipped = LQRController(env_linear_clipped, dynamics_linear, Q_0, R, freq, name='LQR_case1_clipped')
# Instantiate the simulator, and then run the simulation
simulator_lqr_linear_clipped = Simulator(dynamics, controller_lqr_linear_clipped, env_linear_clipped, 1/freq, t_terminal)
simulator_lqr_linear_clipped.run_simulation()
# Instantiate the visualizer, and display the plottings and animation
visualizer_lqr_linear_clipped = Visualizer(simulator_lqr_linear_clipped)
visualizer_lqr_linear_clipped.display_contrast_plots(simulator_lqr_linear)
visualizer_lqr_linear_clipped.display_contrast_animation_same(simulator_lqr_linear)



#### **Results Analysis**

1. **Input Response**:  
   - In the input plot (right panel), the ideal LQR (red dashed line) generates a continuous and reasonable input.  
   - However, the clipped LQR (blue solid line) shows clear cut-off behavior, where the input is clipped to the specified upper and lower bounds (gray shaded regions). This results in a flattened profile that deviates from the ideal input.

2. **Position Response**:  
   - In the position plot (left panel), the ideal trajectory (LQR_0, red dashed line) smoothly reaches the target position.  
   - In contrast, the clipped trajectory (LQR_real_traj, blue solid line) shows slower convergence due to input clipping.

Despite the suppression of velocity by the input constraints, the car is still able to gradually reach the target position. This is attributed to the fact that, for linear systems, a stable equilibrium point is typically globally attractive; which means that, the system will asymptotically converge to the target state from any initial condition. **However, such a property does not generally hold for a nonlinear system, as demonstrated in the following example.**


To further illustrate this limitation for the nonlinear systems, we design following two test cases:

- **Ideal LQR Control for Nonlinear System (`LQR_nonlinear`)**:  

   - Nonlinear system 

   - Linearization point for LQR is chosen to be $\overline{x} = [0, 0]$ (on the slope)

   - No constraints are applied to the control inputs.

- **Clipped LQR Control for Nonlinear System (`LQR_nonlinear_clipped`)**: 

   - Input constraints are imposed, which means that the input is limited within an upper and lower bound

   - Any control input exceeding these bounds is **clipped**, leading to an underactuated system

In [ ]:
# Define the case
case_nonlinear = 4

# Instantiate class 'Env'
env_nonlinear = Env(case_nonlinear, np.array([initial_position, initial_velocity]), np.array([target_position, target_velocity]))
# Instantiate class 'Dynamics'
dynamics_nonlinear = Dynamics(env_nonlinear)
# Instantiate the LQR controller class
controller_lqr_nonlinear = LQRController(env_nonlinear, dynamics_nonlinear, Q_1, R, freq, name='LQR_case4')
controller_lqr_nonlinear.set_lin_point(np.array([0.0, 0.0]))
# Instantiate the simulator, and then run the simulation
simulator_lqr_nonlinear = Simulator(dynamics_nonlinear, controller_lqr_nonlinear, env_nonlinear, 1/freq, t_terminal)
simulator_lqr_nonlinear.run_simulation()



# Define a upper bound of input a
input_ubs = 8.0
input_lbs = -5.0

# Instantiate class 'Env' and visualize the shape of the slope (left side) and theta curve (right side) 
env_nonlinear_clipped = Env(case_nonlinear, np.array([initial_position, initial_velocity]), np.array([target_position, target_velocity]), 
                           input_lbs=input_lbs, input_ubs=input_ubs)
# Instantiate the LQR controller class
controller_lqr_nonlinear_clipped = LQRController(env_nonlinear_clipped, dynamics_nonlinear, Q_1, R, freq, name='LQR_case4_clipped')
controller_lqr_nonlinear_clipped.set_lin_point(np.array([0.0, 0.0]))
# Instantiate the simulator, and then run the simulation
simulator_lqr_nonlinear_clipped = Simulator(dynamics_nonlinear, controller_lqr_nonlinear_clipped, env_nonlinear_clipped, 1/freq, t_terminal)
simulator_lqr_nonlinear_clipped.run_simulation()
# Instantiate the visualizer, and display the plottings and animation
visualizer_lqr_nonlinear_clipped = Visualizer(simulator_lqr_nonlinear_clipped)
visualizer_lqr_nonlinear_clipped.display_contrast_plots(simulator_lqr_nonlinear)
visualizer_lqr_nonlinear_clipped.display_contrast_animation(simulator_lqr_nonlinear)



#### **Results Analysis**

1. **Input Response**:  
   - In the input plot (right panel), the ideal LQR (red dashed line) generates a reasonable input that a larger control effort is required when the system is far from the target, whereas the required input decreases as the system approaches the target.
   - However, the clipped LQR (blue solid line) shows clear cut-off behavior, where the input is clipped to (and consistently bounded to) the specified upper (gray shaded regions).

2. **Position Response**:  
   - In the position plot (left panel), the ideal trajectory (LQR_0, red dashed line) smoothly reaches the target position. However, due to system nonlinearity, it fails to stabilize at the target and quickly diverges from the equilibrium point.
   - In contrast, the clipped trajectory (LQR_real_traj, blue solid line) fails to reach the target position due to input constraints, which prevent it from generating sufficiently large control inputs to overcome the slope. As a result, it keeps oscillating at the bottom of the valley.

This limitation highlights the importance of considering input constraints during controller design. Methods such as **Model Predictive Control (MPC)** can address this issue by explicitly incorporating input limits into the optimization problem.